In [13]:
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='52KYU1B4HW8FCKX0')

In [14]:
class ScriptData:

    def __init__(self):
        self.fetched_data = {}
        self.converted_data = {}

    def __getitem__(self, script, datatype='converted'):
        if datatype == 'converted':
            try:
                return self.converted_data[script]
            except:
                return 'converted intraday data not available'
        elif datatype == 'fetched':
            try:
                return self.fetched_data[script]
            except:
                return

    def __setitem__(self, script, data, datatype):
        if datatype == 'fetched':
            self.fetched_data[script] = data
        elif datatype == 'converted':
            self.converted_data[script] = data

    def __contains__(self, script):
        if script in self.converted_data:
            return True
        return False

    def fetch_intraday_data(self, script):
        data, metadata = ts.get_intraday(symbol=script)
        self.__setitem__(script, data, 'fetched')

    def convert_intraday_data(self, script):
        data = self.__getitem__(script, 'fetched')
        if data is None:
            return 'fetch data for this stock first'
        df = pd.DataFrame(data).T
        df.columns = ['open','high','low','close','volume']
        df.insert(0, 'timestamp', df.index)
        df.index = np.arange(len(data))
        self.__setitem__(script, df, 'converted')

In [15]:
script_data = ScriptData()

In [16]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,94.2400,94.2400,93.5000,94.1200,27980
1,2023-02-14 19:45:00,94.2700,94.3200,94.2000,94.2000,11286
2,2023-02-14 19:30:00,94.3400,94.3400,94.2500,94.3000,2459
3,2023-02-14 19:15:00,94.4500,94.5000,94.2600,94.2600,13234
4,2023-02-14 19:00:00,94.3000,94.4900,94.3000,94.3600,14382
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,94.2900,94.4300,94.2200,94.3950,1182749
96,2023-02-13 12:00:00,94.3064,94.3600,94.1200,94.3000,1523977
97,2023-02-13 11:45:00,94.4600,94.5000,93.9800,94.3021,2155180
98,2023-02-13 11:30:00,94.6500,94.8000,94.3800,94.4500,1425631


In [17]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,152.7500,152.8500,152.7000,152.8400,17795
1,2023-02-14 19:45:00,153.0100,153.0200,152.7500,152.7600,15033
2,2023-02-14 19:30:00,152.8500,153.0300,152.8200,153.0300,10460
3,2023-02-14 19:15:00,153.0800,153.0900,152.8400,152.8400,9621
4,2023-02-14 19:00:00,153.0800,153.0900,153.0700,153.0900,1409
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,153.4400,153.8780,153.4100,153.8300,1507638
96,2023-02-13 12:00:00,153.5100,153.6900,153.2800,153.4200,1741812
97,2023-02-13 11:45:00,153.1500,153.5200,152.9678,153.5100,1815044
98,2023-02-13 11:30:00,152.8940,153.2100,152.8940,153.1423,2038315


In [18]:
'GOOGL' in script_data

True

In [19]:
'AAPL' in script_data

True

In [20]:
'NVDA' in script_data

False